# Hvordan fordele eiendeler med utgangspunkt i ønskelister?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import cvxopt

## Maksimal flyt med Edmonds Karp

In [2]:
users = ['A', 'B', 'C']
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wishlists

[array([0, 2, 4, 1, 3]), array([1, 2, 0, 3, 4]), array([1, 0, 2, 4, 3])]

In [3]:
class Edge():
    name = ""
    flows = {}
    capacities = {}
    
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name + ": " + str(self.edges)
    
user_nodes = [Node(u) for u in users]
asset_nodes = [Node(a) for a in assets]
s = Node("s")
t = Node("t")

NameError: name 'Node' is not defined

In [ ]:
for i in range(len(users)):
    user_nodes[i].edges = {asset_nodes[j]: [0, wishlists[i][j]] for j in range(len(assets))} | {s: [0, len(assets)]}
    
for i in range(len(assets)):
    asset_nodes[i].edges = {user_nodes[j]: [0, wishlists[j][i]) for j in range(len(users))} | {t: (0, len(assets))}
    
s.edges = {u_n: (0 len(assets)) for u_n in user_nodes}
t.edges = {a_n: len(assets) for a_n in asset_nodes}

In [ ]:
flows = {(u_n, a_n): 0 for u_n, a_n in itertools.product(user_nodes, asset_nodes)}\
        | {(s, u_n): 0 for u_n in user_nodes}\
        | {(a_n, t): 0 for a_n in asset_nodes}

In [ ]:
edges = list(flows.keys())

In [ ]:
flow = 0
while True:
    queue = [s]
    visited = []
    while len(q) > 0:
        current = queue.pop()
        for edge in current.edges:
            

Etter litt refleksjon har jeg kommet på at løsninger på maksimal flyt kan gi fordelinger der enkelteiendeler må deles mellom brukere. Dette er upraktisk i praksis. Vi kan derfor ikke bruke maksimal flyt.

## 0-1 Integer Lineær programmering
"Minimize c*x subject to Ax <=b, x ∈ {0, 1}"

In [21]:
relation_weights = {
    'child': 1,
    'parent': 0.8,
    'sibling': 0.7,
    'grandchildren': 0.6,
    'grandparent': 0.4,
    'pibling': 0.15,
    'other': 0.1
}

relation_priority = ['child', 'parent', 'sibling', 'grandchildren', 'grandparent', 'pibling', 'other']

In [22]:
users = ['A', 'B', 'C']
relations = np.random.choice(relation_priority, len(users))
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wish_hstack = np.hstack(wishlists) - len(assets)
c = np.array([wish_hstack[i] * relation_weights[relations[i//len(assets)]] for i in range(len(wish_hstack))])
c

array([-0.15, -0.6 , -0.45, -0.3 , -0.75, -2.1 , -1.4 , -0.7 , -2.8 ,
       -3.5 , -0.45, -0.15, -0.6 , -0.75, -0.3 ])

In [23]:
relations

array(['pibling', 'sibling', 'pibling'], dtype='<U13')

In [24]:
weight_factor = 1 / max(relation_weights[relation] for relation in relations)
weight_factor

1.4285714285714286

In [33]:
wishlists

[array([4, 1, 2, 3, 0]), array([2, 3, 4, 1, 0]), array([2, 4, 1, 0, 3])]

In [25]:
max_one_user_per_asset = [[1 if (i-j)%len(assets)==0 else 0 for i in range(len(c))] for j in range(len(assets))]
fair_distribution = [[wish_hstack[i] if len(assets)*j <= i < len(assets)*(j+1) else 0 for i in range(len(c))] for j in range(len(users))]
A = np.array(max_one_user_per_asset + fair_distribution)
A

array([[ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1],
       [-1, -4, -3, -2, -5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -3, -2, -1, -4, -5,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -3, -1, -4, -5, -2]])

In [26]:
max_one_user_per_asset_sum = [1] * len(assets)
fair_distribution_sum = [wish_hstack[:len(assets)].sum() * relation_weights[relation] * weight_factor / len(users) for relation in relations]
b = np.array(max_one_user_per_asset_sum + fair_distribution_sum)
b

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
       -1.07142857, -5.        , -1.07142857])

In [27]:
A.shape

(8, 15)

In [28]:
G = cvxopt.matrix(A, tc='d')
G

<8x15 matrix, tc='d'>

In [29]:
h=cvxopt.matrix(b, tc='d')
h

<8x1 matrix, tc='d'>

In [30]:
from cvxopt.glpk import ilp

solution = ilp(c=cvxopt.matrix(c, tc='d'),
               G=cvxopt.matrix(A, tc='d'),
               h=cvxopt.matrix(b, tc='d'),
               B=set(range(len(c))))
solution

('optimal', <15x1 matrix, tc='d'>)

In [31]:
opt = np.array(list(solution[1]))
opt

array([0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.])

In [32]:
np.matmul(A, opt)

array([  1.,   1.,   1.,   1.,   1.,  -4., -12.,  -4.])

Litt testing og logikk tilsier at denne algoritmen er optimal og rettferdig. Vi går derfor for binær linear programmering som løsning.

## Testing av full_distribution()

In [2]:
import os, sys
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
sys.path.insert(0, os.path.abspath("../backend"))
os.environ["DJANGO_SETTINGS_MODULE"] = "backend.settings"

In [3]:
import django
django.setup()

In [4]:
from roddi.models import *

In [5]:
from roddi.test import setup_test_data
setup_test_data.main()

In [6]:
estate = Estate.objects.all().get()
users = User.objects.all()

In [7]:
estate.full_distribution()

In [8]:
[Relation.objects.all().get(user=u).relation for u in users]

['pibling', 'sibling', 'parent']

In [9]:
[(user, user.get_ordered_wishlist()) for user in users]

[(<User: Daniel>,
  [<Asset: PC>,
   <Asset: Vase>,
   <Asset: Gevær>,
   <Asset: Bord>,
   <Asset: Stol>,
   <Asset: Maleri>]),
 (<User: Philip>,
  [<Asset: Vase>,
   <Asset: Maleri>,
   <Asset: Bord>,
   <Asset: Stol>,
   <Asset: Gevær>,
   <Asset: PC>]),
 (<User: Steffen>,
  [<Asset: Maleri>,
   <Asset: Gevær>,
   <Asset: Bord>,
   <Asset: PC>,
   <Asset: Stol>,
   <Asset: Vase>])]

In [10]:
[(user, user.obtained_assets.all()) for user in users]

[(<User: Daniel>, <QuerySet [<Asset: Bord>]>),
 (<User: Philip>, <QuerySet [<Asset: Vase>, <Asset: Maleri>, <Asset: Stol>]>),
 (<User: Steffen>, <QuerySet [<Asset: Gevær>, <Asset: PC>]>)]

In [11]:
estate.is_complete

True

Alt ser ut til å fungere

In [14]:
?ilp